In [ ]:
import openai
import pandas as pd
import polars as pl

openai.api_key = ''

In [ ]:
def completion(model_name: str, prompt: str) -> dict:
    """Get a completion from the fine-tuned model.

    Args:
        model_name (str): name of the fine-tuned model.
        prompt (str): prompt to be completed.

    Returns:
        response (dict): response from the OpenAI API.
    """
    response = openai.Completion.create(
        model=model_name,
        prompt=prompt
    )
    return response

In [ ]:
# listando os modelos treinados
models = openai.Model.list()
model_name = models['data'][-1]['id']

In [ ]:
# importando os dados de teste
test = pl.read_csv('../datasets/gpt/tv/test.csv')
prompts = [s for s in test['review'].to_list()]

In [ ]:
# buscando os resultados
responses = []
for prompt in prompts:
    response = completion('ft:gpt-3.5-turbo-0613:personal::7rSxQMOV', prompt)
    responses.append(response['choices'][0]['text'])

In [ ]:
# limpando a resposta. nesse caso, a resposta termina no ".".
# conforme a estrutura do dado de treinamento
# (ver /datasets/gpt/tv/train_prepared.jsonl).
def clean_response(response):
    response = response.split('.')[0]
    response = response.split('\n')
    response[0] = response[0][1:]
    return response

responses_cleaned = [clean_response(r) for r in responses]

In [ ]:
# salvando os resultados
test = pd.read_csv('../datasets/gpt/tv/test.csv')
test['curie_results'] = responses_cleaned

In [ ]:
# gravando o resultado
test.to_csv('../datasets/gpt/tv/test.csv', index=False)